## Example: Twitter API in Python, with OAuth1 authentication

<img src="images/Python_Twitter.jpg" width="300px">

The guide below shows the steps of registering an application that you can integrate with **Twitter**. Registering the application is the first step in developing an application that is integrated into its users' social graph.

## Register a new application

All **Twitter** users are potentially **Twitter** application developers. Simply visit https://apps.twitter.com/ and sign in with your **Twitter** credentials.

Click the "Create a new application" button near the top. A new page with the **_Create an application_** form requires basic information about your application.

<img src="images/create_an_application_1.jpg">

- In the **_Name_** field name your application in 32 characters or fewer. 
- In the **_Description_** field describe your application in 10 to 200 characters.
- In the **_Website_** field, you can give a pointer to your website. (It does not really matter, as we will not be using that functionality yet).
- The **_Callback URL_** leave it empty for now.

The **Developer Agreement** section outlines rules you must agree to follow if you build an application that uses **Twitter’s** API. If you agree to the rules, check "Yes, I agree".

<img src="images/create_an_application_2.jpg">

After Captcha challenge click "Create your **Twitter** application" to complete the form and go to the application settings page.

In the opened window you’ll then be presented with lots of information. The main fields to note are **_Consumer key_** and **_Consumer secret_**. These values are your application’s credentials for **Twitter**. You need them to do almost anything with **Twitter**, including going through the OAuth authorization flow and working with **Twitter’s** REST API. But we’re not quite done yet. We now need to authorise the **Twitter** app for your **Twitter** account. To do this you should select "Key and Access Tokens" menu button and create your access token. This **access token** will allow your **Twitter** application to read **Twitter** information. You’ll be able to get data of your tweets, mentions, lists and more. 

<img src="images/create_an_application_3.jpg">

As a result you will see the window with consumer and token keys for your application 

<img src="images/create_an_application_4.jpg">

Above the four fields are highlighted. You will need these long horrible strings of characters for your **Twitter** app. It goes without saying that you should keep these secret. If anyone was to get these keys, they could effectively access your **Twitter** account.

However, if you want to do more advanced stuff like sending tweets or deleting, you’ll need to change your access type in the "Permission" menu window. Change the access type to "Read and Write" to be able to read **Twitter** data and send tweets and select "Read, Write and access direct messages" if you want to also have access to your direct messages.

---
## Getting access to Twitter APIs using main Python tools
---

Many web services including **Twitter** require authentication, and there are many different types. 

Python library `requests` provides a pythonic way to make complex HTTP requests, and handles difficult tasks like authentication.

A common form of authentication for several web APIs is OAuth. The `requests-oauthlib` library allows `requests`'s users to easily make OAuth authenticated requests.

In [ ]:
!sudo -H pip3 install -U requests_oauthlib

In [ ]:
import requests
from requests_oauthlib import OAuth1

import json

# Let's define consumer and access keys and secrets for getting access to Twitter API through your application
consumer_key = 'ZFSlBaBhgAnoI5Cb5YJ8Q2iSL'
consumer_secret = 'QBFuxdiXsGypn1fEL3Nu8BQtYiFXC97RIsaCf6xv7Rq7E4Dcyg'
access_token = '16976496-hhbdqyfoVqgYbFmVz076IwWxa4vM58my0v1nJNlW0'
access_secret = '3aTWUUdNMTVWDBjxCOApzRdSiiySFM4yaZcGJ1YYaub75'

# You will authenticate yourself using OAuth1 object
auth = OAuth1(consumer_key, consumer_secret, access_token, access_secret)
print(auth)

In [ ]:
# If the authentication was successful, you should see the name of the account print(out)
url_0 = 'https://api.twitter.com/1.1/account/verify_credentials.json'
res = requests.get(url_0, auth=auth)

print("My name is", res.json()["name"])


In [ ]:
res.json()

### Example 1: Read Tweets from Your Homepage

The list of all available **Twitter** API you can find here https://dev.twitter.com/rest/public. Let's use `home_timeline` that returns a collection of the most recent tweets and retweets posted by the authenticating user and the users they follow.

In [ ]:
url_1 = 'https://api.twitter.com/1.1/statuses/home_timeline.json'
res = requests.get(url_1, auth=auth, params={"count": 20})

# The res object encapsulates the "response" of the server. Notice the status code that is displayed. 
# Code 200 means that things went fine
# Code 403 means that the server understood the request, but is refusing to fulfill it
# Code 404 means that the URL was not found
# Codes 5xx mean that something went wrong

print(res, "Status code:", res.status_code)
# Let's see how looks the url
print(res.url)
print("Content type:", res.headers['content-type'], '\n')

tweets = res.json()
if res.status_code == 200:
    for tweet in tweets:
        print("* ", tweet['text'])
        print("----------------------------------")
else:   # You have no tweets
    pass

### Example 2: Search tweets by key words

**Twitter** employs a special query language.  For example, the query _"vacation?"_ will return tweets that contain the word "vacation" and are phrased as a question OR the query _"summer :)"_ will return the word "summer" with a positive attitude. [Check out more examples here](https://dev.twitter.com/rest/public/search).

We can mine tweets using either **_search_** or **_stream_**.

>The key difference between **_stream_** and **_search_** is that **_stream_** provides new data as it comes in, while **_search_** can be used to query old data. The **_search_** API is more powerful for queries, and provides faster access to a wide-range of data.

Let's search for a single tweet about "vacation", phrased as a question. Note if request phrase contains URI characters you should [encode](https://en.wikipedia.org/wiki/Percent-encoding) these queries before making the request, because these characters can play role of keys in an URL. Particularly, the question mark "?" is encoded as "%3F".

In [ ]:
import urllib

query = "stern,nyu"
encoded_query = urllib.parse.quote(query)

url_2 = 'https://api.twitter.com/1.1/search/tweets.json?q=' + encoded_query
res = requests.get(url_2, auth=auth)

print(res, res.status_code, res.headers['content-type'])
print(res.url)

tweets = res.json()
tweets

Please pay attention on how much information each tweet contains.

In [ ]:
# And let's see tweets text directly
for num, tweet in enumerate(tweets['statuses']):
    print('Tweet #{0}\t{1}\t{2}\n{3}\n'.format(num+1, tweet['created_at'], 
                                               tweet['retweet_count'], 
                                               tweet['text'].encode("utf-8")))

In [ ]:
import pandas as pd
df = pd.DataFrame(tweets['statuses'])
df

##### A small diversion: The `json_normalize` function in Pandas

In [ ]:
mylist = [
    {"FullName": {
            "Given": {
                "First": "Panos", 
                "Middle": "Gregory"
            }, 
            "Family": "Ipeirotis"
        }, 
     "Age": 40
    },
    {"FullName": {
            "Given": {
                "First": "John", 
                "Middle": "Reuben"
            }, 
            "Family": "Smith"
        }, 
     "Age": 25
    },
    {"FullName": {
            "Given": {
                "First": "Jane", 
                "Middle": "Janet"
            }, 
            "Family": "Smith"}, 
     "Age": 22
    }
]

df_test = pd.DataFrame(mylist)
df_test

In [ ]:
from pandas.io.json import json_normalize
df_test_normalized = json_normalize(mylist)
df_test_normalized

##### end of diversion... and using the function to make our JSON results more readable

In [ ]:
# If we want to flatten the nested JSON entries
# we can use the json_normalize function
# http://pandas-docs.github.io/pandas-docs-travis/io.html#normalization
import pandas as pd
from pandas.io.json import json_normalize
df_flat = json_normalize(tweets['statuses'])


You may create very specialized and more concrete queries. 

Let's find 5 tweets that contains the word "python" or "IPython" near New York. We can provide this as a `geocode` with a lattitude, longitude and radius. We can also specify time range of tweets appearance (with the help of `since` and `until` key words) and the tweet language (the `lang` parameter restricts tweets to the given language). Additional parameters allows build very complicated search requests.

In [ ]:
url_2_2 = 'https://api.twitter.com/1.1/search/tweets.json'

params = {
    "count": 10, 
    "geocode": '40.7127,-74.0059,15mi',
    "lang": 'en',
    "q": 'python'
}

res = requests.get(url_2_2, auth=auth, params=params)

print(res, res.status_code, res.headers['content-type'])
print(res.url)

tweets = res.json()

for num, tweet in enumerate(tweets['statuses']):
    print("-----------------------------")
    print('Tweet )#{0}\t{1}\n{2}\n'.format(num+1, tweet['created_at'], tweet['text'].encode("utf-8")))

### Example 3: Get a list of all your followers

**Twitter** GET request `followers/list` returns a cursored collection of user objects for users following the specified user. At this time, results are ordered with the most recent following first.

In [ ]:
url_3 = 'https://api.twitter.com/1.1/followers/list.json'
res = requests.get(url_3, auth=auth)
print(res, res.status_code, res.headers['content-type'])
print(res.url)

followers = res.json()
followers

In [ ]:
# And let's see only followers' name
for num, follower in enumerate(followers['users']):
    print('My follower #{0}\t{1}'.format(num+1, follower['name'].encode("utf-8")))
    

In [ ]:
# And now an example of using a "cursor" to go through pages of results
cursor = res.json().get("next_cursor_str")
res = requests.get(url_3, auth=auth, params={"cursor": cursor})
print(res, res.status_code, res.headers['content-type'])
print(res.url)

followers = res.json()
for num, follower in enumerate(followers['users']):
    print('My follower #{0}\t{1}'.format(num+1, follower['name'].encode("utf-8")))
    


### Example 4:  Finding what is trending

According to the [tweepy API](http://tweepy.readthedocs.org/en/v3.5.0/api.html), we can return the top 50 trending topics for a specific location, where the location is a `WOEID (Yahoo Where on Earth ID)`. 

The `WOEID` is a unique identifier, similar to zipcodes, but that expand worldwide. For example, New York has a `WOEID` of 2459115. You can search for `WOEID`'s here: http://woeid.rosselliot.co.nz/.

Let's see the top trending topics in New York

In [ ]:
url_4 = 'https://api.twitter.com/1.1/trends/place.json?id=2459115'
res = requests.get(url_4, auth=auth)

print(res, res.status_code, res.headers['content-type'])
print(res.url)

top50_trends = res.json()
top50_trends

In [ ]:
# And let's see only the first 10 trends names and the respective URL
for i, trend in enumerate(top50_trends[0]['trends'][:10]):
    print('{0} - {1} - URL: {2}'.format(i+1, trend['name'], trend['url']))

### Example 5: Streaming

**Twitter** offers a [Streaming API](https://dev.twitter.com/streaming/overview) to make it easier to query streams of tweets.  The Stream API encapsulates some pain points of REST access to ensure that Stream calls don't exceed the rate limit. These tool allows to get tweet data when it appears in real time. There are three stream types:

- `Public Streams:` Streams of public data flowthing through **Twitter**. Suitable for followign specific users, topics or for data mining.
- `User Streams:` Single-user streams. Containing roughly all of the data corresponding with a single user's view of **Twitter**. 
- `Site Streams:` The multi-user version of user streams.

>Connecting to the streaming API requires keeping a persistent HTTP connection open. In many cases this involves thinking about your application differently than if you were interacting with the REST API. An app which connects to the Streaming APIs will not be able to establish a connection in response to a user request. Instead, the code for maintaining the Streaming connection is typically run in a process separate from the process which handles HTTP requests. The streaming process gets the input tweets and performs any parsing, filtering, and/or aggregation needed before storing the result to a data store. The HTTP handling process queries the data store for results in response to user requests. While this model is more complex than the first example, the benefits from having a realtime stream of tweet data make the integration worthwhile for many types of apps.

With `requests.Response.iter_lines()` you can easily iterate over streaming APIs including the **Twitter Streaming API**. Simply set `stream` parameter to `True` and iterate over the response with `iter_lines()`. 

Depending on the search term/terms, we can get tons of tweets within a few minutes.

A working example that gathers all the new tweets with the _#iphone_ hashtag:

In [ ]:
import time
import urllib

query = "#iphone"
encoded_query = urllib.parse.quote(query)
url_5 = 'https://stream.twitter.com/1.1/statuses/filter.json?track=' + encoded_query
res = requests.get(url_5, auth=auth, stream=True)

print(res, res.status_code, res.headers['content-type'])
print(res.url, "\n")

# Let's measure the time elapsed after the start of streaming 
start = time.time()
stop_after = 10
tweets_printed = 0
for line in res.iter_lines():
    # filter out keep-alive new lines
    if line:
        parsed_line = json.loads(line.decode(encoding='utf-8') )
        print("Elapsed:", time.time()-start, "sec",  
                parsed_line["created_at"], "\n", 
                parsed_line['text'].encode("utf-8"), "\n")
        tweets_printed = tweets_printed + 1
        if tweets_printed>stop_after:
            break

print("Time Elapsed for getting", stop_after, " tweets about ", query, " ==>" , time.time()-start)

---
## Exercises
---

## Exercise #1: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library display, the list of all your friends and sort them by its followers amounts in descending order. 

#### HINT: `GET friends/list`  will be helpfull for you here.

## Exercise #2: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library, display the list of 5 (or less if there is not so much) most recent tweets liked by you, which were also retweeted (`retweet_count` is not equal to zero). You shoud show the date of creation, author of the tweet and its text.

#### HINT: `GET favorites/list` will be helpfull for you here

## Exercise #3: 

> Using [Twitter Streaming API](https://dev.twitter.com/streaming/overview) and `requests` Python library count how many tweets with hashtags _#twitter_, _#tweet_ and _#world_ are appearing each minute over 5 minutes and display these 5 numbers. Please display also the shortest and the longest time period between to closest tweets for all 5 measures.

#### HINT: `POST statuses/filter` will be helpfull for you here